# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 3-month treasury bond

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=32

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==32]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = df3['interest_rate']/100
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_ttm32', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_ttm32/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-08-28 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"


...

[1] 44.694713  4.722982 66.258718
            m      spdy   spdy_up   spdy_dn       epk       ret   spd_ret  \
0    0.514218  0.125315  0.206501  0.044130  0.418259 -0.665109  0.064439   
1    0.524098  0.135984  0.226980  0.044987  0.400233 -0.646077  0.071269   
2    0.533978  0.147564  0.249130  0.045997  0.386224 -0.627400  0.078796   
3    0.543858  0.160130  0.273038  0.047221  0.375579 -0.609066  0.087088   
4    0.553739  0.173760  0.298791  0.048728  0.367779 -0.591062  0.096217   
..        ...       ...       ...       ...       ...       ...       ...   
195  2.440871  0.001899  0.034476 -0.030678       inf  0.892355  0.004635   
196  2.450752  0.002240  0.034172 -0.029693       inf  0.896395  0.005489   
197  2.460632  0.002578  0.033859 -0.028702       inf  0.900418  0.006344   
198  2.470512  0.002914  0.033536 -0.027709       inf  0.904426  0.007198   
199  2.480392  0.003244  0.033203 -0.026714       inf  0.908417  0.008047   

        cb_ret_up     cb_ret_dn       epk

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
            m      spdy   spdy_up   spdy_dn           epk       ret   spd_ret  \
0    0.535738  0.166067  0.187259  0.144876  4.258775e-01 -0.624110  0.088969   
1    0.542678  0.173760  0.196810  0.150709  4.126694e-01 -0.611239  0.094296   
2    0.549618  0.181873  0.206880  0.156867  4.013922e-01 -0.598532  0.099961   
3    0.556558  0.190431  0.217490  0.163371  3.918341e-01 -0.585984  0.105986   
4    0.563498  0.199457  0.228667  0.170247  3.838122e-01 -0.573592  0.112393   
..        ...       ...       ...       ...           ...       ...       ...   
195  1.889022  0.002447  0.024351 -0.019456  1.284056e+10  0.636059  0.004623   
196  1.895962  0.002267  0.024424 -0.019890  2.928335e+10  0.639726  0.004298   
197  1.902902  0.002101  0.024507 -0.020305  6.792820e+10  0.643380  0.003999   
198  1.909842  0.001950  0.024601 -0.020700  1.588614e+11  0.647020  0.003725   
199  1.916782  0.001814  0.024704 -0.021076  3.684943e+11  0.650648  0.0034

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_84994/1724297816.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
            m      spdy   spdy_up   spdy_dn           epk       ret   spd_ret  \
0    0.518415  0.063697  0.185794 -0.058399  4.199781e-01 -0.656979  0.033022   
1    0.526206  0.068641  0.199058 -0.061775  3.989520e-01 -0.642063  0.036119   
2    0.533996  0.073977  0.213039 -0.065085  3.799594e-01 -0.627366  0.039503   
3    0.541787  0.079735  0.227769 -0.068300  3.626501e-01 -0.612882  0.043199   
4    0.549578  0.085948  0.243281 -0.071386  3.468447e-01 -0.598605  0.047235   
..        ...       ...       ...       ...           ...       ...       ...   
195  2.037584 -0.001657  0.041252 -0.044565          -inf  0.711765 -0.003376   
196  2.045375 -0.001683  0.040846 -0.044213 -1.616793e+14  0.715581 -0.003443   
197  2.053165 -0.001707  0.040447 -0.043860          -inf  0.719383 -0.003504   
198  2.060956 -0.001728  0.040053 -0.043509 -8.979954e+13  0.723170 -0.003561   
199  2.068747 -0.001746  0.039665 -0.043158          -inf  0.726943 -0.0036

In [2]:
print(error_i)

[0, 5, 6, 10, 15]
